In [5]:
import requests
import json
import hashlib
import pandas as pd
import datetime
import string
from pprint import pprint as pp
import time
from pandas import json_normalize
import os

In [6]:
ts = datetime.datetime.now()
def dataframe_generator(nameStartsWith,APIKEY=None,Hash=None):
    global df
    if APIKEY is None or Hash is None:
        raise Exception("Expected arguments!!!")
    else:
        request_url = 'http://gateway.marvel.com/v1/public/characters'
        params ={'ts':ts,'apikey':APIKEY,'hash':Hash,
          'nameStartsWith':nameStartsWith,'limit':100}
        response = requests.get(request_url,params=params)
        data_1 =response.json()
        df = pd.json_normalize(data_1,record_path=['data','results'])
        df_new=df[['name','id','comics.available','stories.available',
                       'events.available','series.available']]
        return df_new



In [7]:
public_key= '2c560d4fa00f2af1802aac30fc3fab93'
private_key= '156a3abb9272f2bf9d301958f307f961f7310c6e'
hash_md5 = hashlib.md5()
hash_md5.update(f'{ts}{private_key}{public_key}'.encode('utf-8'))
hash_params = hash_md5.hexdigest()
desired_df=dataframe_generator(nameStartsWith='A',APIKEY='2c560d4fa00f2af1802aac30fc3fab93',
                      Hash=hash_params)
desired_df.head()


,name,id,comics.available,stories.available,events.available,series.available
0,A-Bomb (HAS),1017100,4,7,0,2
1,A.I.M.,1009144,51,54,0,34
2,Aaron Stack,1010699,14,27,0,3
3,Abomination (Emil Blonsky),1009146,55,65,1,27
4,Abomination (Ultimate),1016823,2,3,0,2


In [3]:
def filter_character(dataframe,column,filter):
    if filter[0]=='>':
        filt_df=dataframe[dataframe[column]>filter[1]]
    if filter[0]=='<':
        filt_df=dataframe[dataframe[column]<filter[1]]
    if filter[0]=='=':
        filt_df=dataframe[dataframe[column]==filter[1]]
    return filt_df

In [8]:
df_based_on_filter=filter_character(desired_df,'comics.available',['<',10])
df_based_on_filter.head()

,name,id,comics.available,stories.available,events.available,series.available
0,A-Bomb (HAS),1017100,4,7,0,2
4,Abomination (Ultimate),1016823,2,3,0,2
6,Abyss,1009149,8,8,1,3
7,Abyss (Age of Apocalypse),1010903,3,2,1,3
8,Adam Destine,1011266,0,0,0,0
